# AI Assistant for Sales Calls: Analyzing and Querying Transcripts

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import month,year,col,sum,count,avg,length,max,min,to_timestamp_ltz

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"ai_assistant_sales_call", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}

# Set session context 
session.use_role("SYSADMIN")

# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

## Data exploration
Data exploration is the process of examining and analyzing datasets to uncover patterns, trends, and anomalies. For sales call transcripts, it helps us understand key metrics like call duration, transcript lengths, and account associations. This foundational step ensures that we have a clear understanding of the data, enabling effective downstream analysis, summarization, and actionable insights for improving sales strategies and customer interactions.

Load Tables, i.e. `df = session.table('table_name')`

In [ ]:
# Load tables directly using session.table
try:
    df_account_d = session.table("ACCOUNT_D")
    print("Successfully loaded ACCOUNT_D")
except Exception as e:
    print(f"Error loading ACCOUNT_D: {e}")

try:
    df_calls_transcript = session.table("CALLS_TRANSCRIPT")
    print("Successfully loaded CALLS_TRANSCRIPT")
except Exception as e:
    print(f"Error loading CALLS_TRANSCRIPT: {e}")

try:
    df_calls = session.table("CALLS")
    print("Successfully loaded CALLS")
except Exception as e:
    print(f"Error loading CALLS: {e}")

try:
    df_gong_gong_call_c = session.table("GONG_GONG_CALL_C")
    print("Successfully loaded GONG_GONG_CALL_C")
except Exception as e:
    print(f"Error loading GONG_GONG_CALL_C: {e}")

### Account Validity by State

This cell groups the account data by ACCOUNT_STATE and ETL_VALID_NOW to calculate the number of valid and invalid accounts for each state. The resulting data is sorted by ACCOUNT_STATE in ascending order and provides insights into the distribution of account validity across different states.

In [ ]:
account_validity_by_state = (
    df_account_d.group_by(col("ACCOUNT_STATE"), col("ETL_VALID_NOW"))
    .agg(count("*").as_("ACCOUNT_COUNT"))
    .sort(col("ACCOUNT_STATE").asc())
)

account_validity_by_state.show()

### Transcript Length Statistics

In this cell, we calculate the follwing summary statistics for the lengths of sales call transcripts:

1. Average length: To understand the typical size of a transcript.
2. Maximum length: To identify the longest transcript.
3. Minimum length: To find the shortest transcript.

This analysis provides insights into the variability of transcript lengths, which can be useful for evaluating the nature and depth of conversations during sales calls.

In [ ]:
transcript_length_stats = (
    df_calls_transcript.select(length(col("TRANSCRIPT")).as_("TRANSCRIPT_LENGTH"))
    .agg(
        avg(col("TRANSCRIPT_LENGTH")).as_("AVG_LENGTH"),
        max(col("TRANSCRIPT_LENGTH")).as_("MAX_LENGTH"),
        min(col("TRANSCRIPT_LENGTH")).as_("MIN_LENGTH")
    )
)

transcript_length_stats.show()

### Late Sales Calls

In this cell, we identify and count the number of sales calls that started later than their scheduled time. We achieve this by comparing the STARTED and SCHEDULED timestamps and filtering for calls where the start time is later than the scheduled time. The result is the total number of late calls, which can help assess the punctuality and efficiency of the sales team.

In [ ]:
calls_started_late = (
    df_calls
    .filter(to_timestamp_ltz(col("STARTED")) > to_timestamp_ltz(col("SCHEDULED")))
    .agg(count("*").as_("LATE_CALL_COUNT"))
)

calls_started_late.show()

### Summarizing Sales Calls by Title
In this cell, we are summarizing the sales calls by their TITLE. We join the df_calls DataFrame with the df_calls_transcript DataFrame using the ID and CALLID columns, respectively. The summary includes the average call duration (CALL_DURATION) and the average transcript length (TRANSCRIPT_LENGTH) for each call title. The result is sorted by CALL_DURATION in descending order to highlight the longest calls. This analysis gives us insights into the duration and length of conversations based on different titles, helping to identify trends and areas for improvement.

In [ ]:
combined_summary = (
    df_calls.join(df_calls_transcript, df_calls["ID"] == df_calls_transcript["CALLID"], how="inner")
    .group_by(col("TITLE"))
    .agg(
        avg(col("DURATION").cast("float")).as_("CALL_DURATION"),
        avg(length(col("TRANSCRIPT"))).as_("TRANSCRIPT_LENGTH"),
    )
    .sort(col("CALL_DURATION").desc())
)

combined_summary.show()

### Visualizing Sales Call Duration and Transcript Length
In this cell, we are visualizing two aspects of the sales calls:

1. Scatter plot: The first plot shows the relationship between CALL_DURATION (in seconds) and TRANSCRIPT_LENGTH (in characters). Each point represents a unique sales call title. This scatter plot helps us visually assess whether longer calls tend to have longer transcripts or if there's no clear correlation.

2. Histogram: The second plot displays the distribution of average call durations. This histogram shows how frequent different call durations are across the dataset, helping to identify trends such as common call lengths or any outliers that might require further analysis.

These visualizations provide a better understanding of the call data and allow us to detect patterns and areas that might benefit from further exploration.

In [ ]:
combined_summary_pd = combined_summary.to_pandas()

# Scatter plot for Call Duration vs Transcript Length (without legend)
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=combined_summary_pd,
    x="CALL_DURATION",
    y="TRANSCRIPT_LENGTH",
    color="blue",
    alpha=0.7,
)
plt.title("Call Duration vs. Transcript Length", fontsize=16)
plt.xlabel("Average Call Duration (seconds)")
plt.ylabel("Average Transcript Length (characters)")
plt.tight_layout()
plt.show()


# Histogram for Call Durations
plt.figure(figsize=(10, 6))
sns.histplot(
    data=combined_summary_pd,
    x="CALL_DURATION",
    bins=20,
    kde=True,
    color="blue",
)
plt.title("Distribution of Average Call Durations", fontsize=16)
plt.xlabel("Average Call Duration (seconds)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

### Grouping Calls by Account

In this step, we are grouping the data by account and counting the number of calls associated with each account to better understand call volume per account.


In [ ]:
# Step 1: Join CALLS with GONG_GONG_CALL_C to associate each call with an account
calls_gong_join = (
    df_calls.join(
        df_gong_gong_call_c,
        df_calls["ID"] == df_gong_gong_call_c["GONG_CALL_ID_C"], 
        how="left"
    )
)

# Step 2: Join the resulting dataframe with ACCOUNT_D to get account details
account_calls_join = (
    calls_gong_join.join(
        df_account_d, 
        df_gong_gong_call_c["GONG_PRIMARY_ACCOUNT_C"] == df_account_d["SFDC_ACCOUNT_ID"],
        how="left"
    )
)

# Step 3: Group by GONG_PRIMARY_ACCOUNT_C and count the number of calls associated with each account
account_call_count = (
    account_calls_join
    .group_by(col("GONG_PRIMARY_ACCOUNT_C"))
    .agg(count("*").alias("CALL_COUNT"))
    .sort(col("CALL_COUNT").desc())
)

# Show the results
account_call_count.show(70)

## Setup for Automated Insights & Analytics

### Run Cortex Sentiment using a SQL cell
In this step, we are using the `SNOWFLAKE.CORTEX.SENTIMENT` function to analyze the sentiment of the transcripts in the `CALLS_TRANSCRIPT` table. The function returns a sentiment score for each transcript, which will help us understand the overall tone of the call. This analysis will be helpful for categorizing calls as positive, negative, or neutral based on the customer interaction.

https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex

In [ ]:
-- Drop columns if they exist
ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS SENTIMENT;

-- Add the 'SENTIMENT' column
ALTER TABLE CALLS_TRANSCRIPT 
ADD COLUMN SENTIMENT FLOAT;

-- Update the 'SENTIMENT' column with the sentiment scores
UPDATE CALLS_TRANSCRIPT
SET SENTIMENT = SNOWFLAKE.CORTEX.SENTIMENT(TRANSCRIPT);

### Run Cortex Summarize using a SQL cell
In this step, we are using the `SNOWFLAKE.CORTEX.SUMMARIZE` function to generate a summary of each call transcript. This will help us quickly grasp the main points of each conversation without having to read through the entire transcript. The summary is useful for identifying key takeaways, trends, and insights from the calls, enabling more efficient data analysis and decision-making.


https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex

In [ ]:
-- Drop columns if they exist
ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS SUMMARY;

-- Add the 'SUMMARY' column
ALTER TABLE CALLS_TRANSCRIPT 
ADD COLUMN SUMMARY STRING;

-- Update the 'SUMMARY' column with the summarized data
UPDATE CALLS_TRANSCRIPT
SET SUMMARY = SNOWFLAKE.CORTEX.SUMMARIZE(TRANSCRIPT);

### Run Cortex Complete using a SQL cell
In this step, we are using the `SNOWFLAKE.CORTEX.COMPLETE` function with a model (`llama3.1-8b`) to extract detailed insights from the call transcripts. We are focusing on three key areas:

1. **Product Feedback**: Extracting both positive and negative feedback on the product, emphasizing specific features, benefits, or issues.
2. **Competitor Information**: Identifying any mentions or comparisons of competitors, including their products/services and strengths/weaknesses.
3. **Sales Activity**: Extracting information about sales-related activities such as deals, pricing, customer purchases, strategies, and follow-up actions.

This process allows us to break down each transcript into valuable, structured insights that can guide decision-making in product development, competitive analysis, and sales strategies.


https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex

In [ ]:
-- Drop columns if they exist
ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS PRODUCT_FEEDBACK;

ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS COMPETITOR_INFORMATION;

ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS SALES_ACTIVITY;

-- Add the columns
ALTER TABLE CALLS_TRANSCRIPT
ADD COLUMN PRODUCT_FEEDBACK STRING, 
            COMPETITOR_INFORMATION STRING, 
            SALES_ACTIVITY STRING;


UPDATE CALLS_TRANSCRIPT
SET 
    PRODUCT_FEEDBACK = SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-70b',
        'Provide the feedback in the following format: \n\nProduct Name: [Product Name or Description]\nPositive Feedback: [Positive feedback here]\nNegative Feedback: [Negative feedback here]. \n\nDo not include any introductory text. Only provide the structured feedback.'
        || TRANSCRIPT
    ),
    COMPETITOR_INFORMATION = SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-70b',
        'Our company is XYZ Corp. Provide the information in the following format: \n\nCompetitor: [Competitor Name]\nCompetitor Service: [Competitor Service Description]\nCompetitor Strength: [Competitor Strength]\nCompetitor weakness: [Competitor Weakness].\n\nDo not include any introductory text. Only provide the structured competitor information.'
        || TRANSCRIPT
    ),
    SALES_ACTIVITY = SNOWFLAKE.CORTEX.COMPLETE(
        'llama3-70b',
        'Provide the information in the following format: \n\n<sales activity>: [Sales activity details]. \n\nDo not include any introductory text. Only provide the structured sales activity information.'
        || TRANSCRIPT
    );

In [ ]:
-- Drop columns if they exist
ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS PRODUCT_MENTIONED;

ALTER TABLE CALLS_TRANSCRIPT 
DROP COLUMN IF EXISTS COMPETITOR_MENTIONED;

-- Add new columns to CALLS_TRANSCRIPT table for analytics
ALTER TABLE CALLS_TRANSCRIPT
ADD COLUMN PRODUCT_MENTIONED STRING,
            COMPETITOR_MENTIONED STRING;

-- Update the new columns with extracted information
UPDATE CALLS_TRANSCRIPT
SET 
    PRODUCT_MENTIONED = REGEXP_REPLACE(
        SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2',
            'You are a product name extractor. Extract only the product name from the following input text. Return the product name as plain text without any prefixes, labels, or formatting.'
            || PRODUCT_FEEDBACK
        ), '\\.$', ''
    ),
    COMPETITOR_MENTIONED = REGEXP_REPLACE(
        SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large2', 
            'You are a competitor name extractor. Extract only the competitor name from the following input text. Return the competitor name as plain text without any prefixes, labels, or formatting. '
            || COMPETITOR_INFORMATION
        ), '\\.$', ''
    );

### Create Cortex Search service

In this step, we are creating a **Cortex Search Service** on the `CALLS_TRANSCRIPT` table. The goal is to enable semantic search capabilities that will allow us to efficiently query and retrieve relevant information from the transcripts. 

- **Attributes**: The search service will use `ID` as the key identifier for each record.
- **Data Transformation**: The `TRANSCRIPT` field is being converted to a string and renamed as `CHUNK` to align with the expected input for the search engine.
- **Search Warehouse**: We specify the warehouse for performing the search operations.

This search service will allow us to perform complex text searches and efficiently analyze the contents of the sales call transcripts.


In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE call_transcript_search_service
  ON CHUNK
  ATTRIBUTES ID
  WAREHOUSE = SALES_CALLS_WH
  TARGET_LAG = '1 day'
  AS (
    SELECT
        CALLID AS ID,
        TRANSCRIPT::STRING AS CHUNK 
    FROM CALLS_TRANSCRIPT
);

In this cell, we use `SNOWFLAKE.CORTEX.SEARCH_PREVIEW` to search for transcripts containing "CompetitorA" in the call_transcript_search_service. The query retrieves the ID and CHUNK columns, with a limit of 1 result. The matching results are parsed to extract the relevant transcript information, helping identify competitor-related content in sales calls.

In [ ]:
SELECT PARSE_JSON(
  SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
     'call_transcript_search_service',
      '{
        "query": "CompetitorA",
        "columns":[
            "ID",
            "CHUNK"
        ],
        "limit":1
      }'
  )
)['results'] as results;